In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance, applyGower

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
cover = loadCoverGroups()

In [4]:
def getTransition():
    transitionMatrices = pd.read_csv(
        './data/unified/07-transitionsClustered.csv')
    transitionMatrices = transitionMatrices[['id', 'category']]
    return transitionMatrices
def getDf():
    df = pd.read_csv(
        './data/unified/05-allDataContinousCategoricalTransition.csv')
    df = df.iloc[:, 2:]
    return df

def getcategoryDF(df,originalSongId):
    categoryOriginalSong = df[df.id == originalSongId].category.values[0]
    df = df[df.category == categoryOriginalSong]
    df = df.drop(columns=['category'])
    # Saving idChords
    ids = df.id
    ids.reset_index(drop=True, inplace=True)
    df = df.set_index('id')
    df.reset_index(drop=True, inplace=True)
    df = removeConstantFeatures(df)
    #df= applyBoxCoxFeatures(df, ['speechiness', 'acousticness', 'liveness', 'valence'])
    #df= applyBoxCoxAllFeatures(df)
    #df = centerAndScale(df)
    scaler = MinMaxScaler()
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df)).copy()
    return (df,ids)

def mergeDF(df, transition):
    df = pd.merge(df, transition, on='id')
    # Removing unamed and other unnecessary columns
    df.reset_index(drop=True, inplace=True)
    df = df.sort_values('id')
    df.id = df.id.astype('int64')
    return df


In [5]:
transitionMatrices = getTransition()
df = getDf()
df = mergeDF(df,transitionMatrices)

In [6]:
def applyGowerAll(df,ids,id,n):
    indexToAnalyze = [i for i,x in enumerate(ids) if x == id][0]
    elementToAnalyze = df.loc[indexToAnalyze:indexToAnalyze,:]
    ranksDFGower = pd.DataFrame(gower.gower_topn(elementToAnalyze, df, n = n))
    songID = pd.DataFrame(ids[ranksDFGower['index']]).reset_index()
    ranksDFGower = ranksDFGower.merge(songID)
    ranksDFGower = ranksDFGower[['id','values']]
    return ranksDFGower

In [11]:
ranksFinalBest = pd.DataFrame(columns=['id','values', 'position', 'set'])
ranksFinalBestCover = pd.DataFrame(columns=['id','values','position', 'set'])
for i, listCovers in enumerate(cover):
    originalSongId = listCovers[0]
    coverSongId = listCovers[1]
    categoryDF,ids =  getcategoryDF(df,originalSongId)
    ranksDFGower = applyGowerAll(categoryDF,ids, originalSongId,len(categoryDF))
    ranksDFGower['position'] = ranksDFGower.index
    ranksDFGower['set'] = i
    coverResults = ranksDFGower.loc[ranksDFGower.id.isin(listCovers)]
    coverResults.reset_index(drop=True,inplace=True)
    if len(coverResults) >= 2:
        ranksFinalBest = pd.concat([ranksFinalBest, ranksDFGower.head(5)])
        ranksFinalBestCover = pd.concat([ranksFinalBestCover, coverResults])

In [12]:
ranksFinalBestCover

,id,values,position,set
0,180849,0.000000,0,0
1,6001027,0.004103,1,0
2,5000017,0.008706,9,0
3,5000019,0.008898,11,0
4,5000001,0.009816,20,0
5,5000020,0.011810,66,0
6,1686718,0.012029,72,0
7,5000013,0.012073,73,0
8,5000018,0.012233,78,0
9,5000011,0.014927,335,0
